# Job Dataset Exploration

In [1]:
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms import user_knn
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

In [2]:
jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)
jobs_df = jobs_data[["JobID", "Title"]]
jobs_df = jobs_df.rename(columns={"JobID": "item", "Title": "title"})

/var/folders/vt/47mxbcvj5rd7mmyk053h49m00000gn/T/ipykernel_64525/1825902803.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)
b'Skipping line 122433: expected 11 fields, saw 12\n'
/var/folders/vt/47mxbcvj5rd7mmyk053h49m00000gn/T/ipykernel_64525/1825902803.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)


In [10]:
# Create function to use in next block
def spaceFilter(df):
    return df["JobIds"]


In [11]:
popular_data = pd.read_csv("dataset/popular_jobs_full.csv", sep=',')
display(spaceFilter(popular_data).head())
display(popular_data.head(10))


0    357049 688334 245041 2102 982331 979937 821993...
1    1105054 1009975 884374 584456 960607 946809 94...
2    1073864 87383 899228 780304 919103 253916 5641...
3    569269 405679 276119 196084 578178 749036 8775...
4    794726 267838 1101979 1097477 1080147 132019 1...
Name:  JobIds, dtype: object

,UserId,JobIds
0,47,357049 688334 245041 2102 982331 979937 821993...
1,72,1105054 1009975 884374 584456 960607 946809 94...
2,80,1073864 87383 899228 780304 919103 253916 5641...
3,98,569269 405679 276119 196084 578178 749036 8775...
4,123,794726 267838 1101979 1097477 1080147 132019 1...
5,131,218261 518070 565974 511687 617448 652233 4642...
6,162,611312 431641 1051911 791614 651211 1062849 82...
7,178,182619 1100604 1006934 306681 157929 639788 15...
8,203,921903 776147 613977 296155 923520 919286 8408...
9,344,901261 886892 601784 901277 554856 334018 5195...


In [5]:
# read the data (about applications)
data = pd.read_csv('dataset/apps.tsv', delimiter='\t')
#display(data.head(10)) # check data being read properly

# extract relevant data
df_ui = data[["UserID", "JobID"]]
#display(df_ui.head(10)) # check data being read properly

# construct dataframe in format (user, item, rating) via column addition
#df_ui.insert(2, "rating", np.random.rand(df_ui.shape[0],1))  
df_ui.insert(2, "rating", np.random.randint(0,2,df_ui.shape[0]))
df_ui = df_ui.rename(columns={"UserID": "user", "JobID": "item"})
display(df_ui.head(10)) # check data being read properly

,user,item,rating
0,47,169528,0
1,47,284009,0
2,47,2121,1
3,47,848187,0
4,47,733748,0
5,47,576958,1
6,47,262470,1
7,47,602298,0
8,72,834662,1
9,72,1020903,0


In [6]:
user_id = 47
# We use the collaborative user algorithm UserUser, that use the nearest neighbors 
user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_ui)

BLAS using multiple threads - can cause oversubscription
found 1 potential runtime problems - see https://boi.st/lkpy-perf


In [7]:
selected_jobs = recsys.recommend(user_id, 100) # generate 10 recommendations for the selected user
display(selected_jobs.head(10))
# score - doesn't matter cause arbitrary value showing recommendation priority
# sometimes ipynb doesn't reset the variables -> gives stupid errors (solution shove everything into one kernel/ block)

selected_jobs = selected_jobs.join(jobs_df.set_index('item'), on='item')
display(selected_jobs.head(10))



,item,score
0,1007701,0.765835
1,456642,0.764273
2,576079,0.749588
3,911239,0.725704
4,265554,0.687492
5,796615,0.679082
6,917560,0.618371
7,1045460,0.575875
8,311769,0.567666
9,927713,0.546626


,item,score,title
0,1007701,0.765835,Insurance Rep (Collector) Temporary
1,456642,0.764273,Assistant General Manager
2,576079,0.749588,Call Center/Sales/ Collections Manager
3,911239,0.725704,ASSISTANT MANAGER
4,265554,0.687492,Customer Order Administrator
5,796615,0.679082,Accounting Clerk
6,917560,0.618371,Administrative Assistant
7,1045460,0.575875,Receptionist/HR Assistant
8,311769,0.567666,Account Payable/Accounts Receivable Coordinator
9,927713,0.546626,Personal Assistant
